<a href="https://colab.research.google.com/github/afrahhassan/afrahhassan/blob/main/assignment2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>



---
# <div align="center"><font>  </font></div>
# <div align="center"><font> COSC 2793 Computational Machine Learning </font></div>
## <div align="center"> <font> Assignment 2: Machine Learning Project </font></div>
## <div align="center"> <font> S3930076 Afrah Hassan A Alshaikh Ali & S3569266 Chang Su </font></div>
---

In [ ]:
#Import Libraries

# **Reading Data**

In [39]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [40]:
import os
os.listdir('/content/drive/MyDrive/Assignment2')


['assignment2_PG.pdf',
 'COSC2673_2793_A2_EOI_.docx',
 'trafficsigns_dataset',
 'trafficsigns_dataset.zip',
 'assignment2.ipynb',
 'images.csv']

In [41]:
!cp -r "/content/drive/MyDrive/Assignment2" .

In [42]:
import zipfile
zip_file_path = '/content/drive/My Drive/Assignment2/trafficsigns_dataset.zip'
extract_to_path = '/content/trafficsigns_dataset'
with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    zip_ref.extractall(extract_to_path)
print("Extraction completed!")

Extraction completed!


In [43]:
import os
import csv

root_dir = '/content/trafficsigns_dataset'
output_csv = '/content/drive/My Drive/Assignment2/images.csv'


with open(output_csv, mode='w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(['image_path', 'class'])

    for root, dirs, files in os.walk(root_dir):
        for file in files:
            if file.endswith(('.png', '.jpg', '.jpeg')):
                full_path = os.path.join(root, file)
                class_name = os.path.basename(root)
                writer.writerow([full_path, class_name])
print("CSV file has been created successfully.")

CSV file has been created successfully.


# export shape to csv



# find each type and shape

#Convert images to numbers

##Baseline Model

##Model Training

##Model Evaluation